In [1]:
%matplotlib inline

from sqlalchemy import create_engine
import datetime
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as col
from matplotlib.dates import  DateFormatter
import seaborn as sns

import config
import  db_service
import data_preparation
import weather_service
import baseline_service

pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)

sns.set_style("whitegrid")
sns.set_context("poster")

from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_absolute_error,explained_variance_score,mean_squared_error,r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble.forest import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import linear_model
from sklearn.grid_search import GridSearchCV

from tpot import TPOTRegressor
from sklearn.model_selection import train_test_split


from sklearn.ensemble import RandomForestRegressor, VotingClassifier
from sklearn.linear_model import LassoLarsCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import FunctionTransformer, MaxAbsScaler

np.set_printoptions(precision=3,suppress=True)

/home/ramon/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/ramon/anaconda2/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
### Call this method to retrieve the data directly from DB :

# db_service.create_counts_sites_weather_holidays_day()

### Otherwise load the file :
df = pd.read_csv('data/counts_sites_weather_holidays_day.csv',parse_dates=['date'],index_col=0)

dataset_past_dict = {}
dataset_forecasts_dict ={}

IOError: File data/counts_sites_weather_holidays_day.csv does not exist

### Data preparation

In [ ]:
# df_forecasting = data_preparation.create_forecasting_data(datetime.date(2016,10,2),datetime.date(2016,10,10))

In [ ]:
start_forecasts = datetime.date(2016,9,1)
end_forecasts = datetime.date(2016,10,9)

In [ ]:
df_past = df[df.date<start_forecasts]
site = 20

In [ ]:
data_past_reg = data_preparation.regularize_dataset(df_past)
data_preparation.create_predictors_and_target(data_past_reg,dataset_past_dict)

df_forecasts = data_preparation.create_forecasts_data(start_forecasts,end_forecasts)

In [ ]:
df_forecasts = data_preparation.regularize_forecast(df_forecasts,dataset_past_dict)

In [ ]:
data_preparation.create_predictors_and_target(df_forecasts,dataset_forecasts_dict, forecasting=True)

In [ ]:
def create_X_Y_per_site(site_id):
    X = dataset_past_dict[site_id]['X'].values
    y = dataset_past_dict[site_id]['y'].values
    fields = dataset_past_dict[site_id]['X'].columns[3:].tolist()

    X_forecast = dataset_forecasts_dict[site_id]['X'].values

    y_forecast = df[(df.date>=start_forecasts)&(df.date<=end_forecasts)&(df.idbldsite==site_id)].compensatedin.values
    return X,y,fields,X_forecast,y_forecast

### Pipeline

In [ ]:
def cv_optimize(clf, parameters, X, y, n_jobs=1, n_folds=5, score_func=None):
    if score_func:
        gs = GridSearchCV(clf, param_grid=parameters, cv=n_folds, n_jobs=n_jobs, scoring=score_func)
    else:
        gs = GridSearchCV(clf, param_grid=parameters, n_jobs=n_jobs, cv=n_folds)
    gs.fit(X, y)
    print "BEST", gs.best_params_, gs.best_score_, gs.grid_scores_
    best = gs.best_estimator_
    return best

In [ ]:
def do_classify(clf, parameters, X,y, mask=None, score_func=None, n_folds=5, n_jobs=1):
    # remove index and idbldsite
    X,y = X[:,3:],y[:,2:].ravel()
    Xtrain, Xtest, ytrain, ytest = train_test_split(X,y,test_size=0.3)
    clf = cv_optimize(clf, parameters, Xtrain, ytrain, n_jobs=n_jobs, n_folds=n_folds, score_func=score_func)
    clf=clf.fit(Xtrain, ytrain)
    training_accuracy = clf.score(Xtrain, ytrain)
    test_accuracy = clf.score(Xtest, ytest)
    
    print "############# based on standard predict ################"
    print "Accuracy on training data: %0.2f" % (training_accuracy)
    print "Accuracy on test data:     %0.2f" % (test_accuracy)
    print "########################################################"
   
#     try:
#         print pd.DataFrame(zip(clf.feature_importances_,fields))
#     except :
#         print "no feature_importance"
    print "########################################################"
    return clf, Xtrain, ytrain, Xtest, ytest

In [ ]:
def compute_forecasting_quality(clf,X,y_true):
    quality_scores = []
    print clf.score(X,y_true)
    y_pred = clf.predict(X)
    quality_scores.append(("mean_absolute_error :" ,mean_absolute_error(y_true,y_pred)))
    quality_scores.append(("Var(MSE) :" ,mean_squared_error(y_true,y_pred)))
    quality_scores.append(("Var(y) : " ,(np.std(y_pred))**2))
    quality_scores.append(("1- mse/y_std : " ,1-(mean_squared_error(y_true,y_pred))/(np.std(y_pred))**2))
    quality_scores.append(("explained_variance_score: " , explained_variance_score(y_true,y_pred)))
    quality_scores.append(("r2_score : " ,r2_score(y_true,y_pred)))
       
    df =  pd.DataFrame(quality_scores,columns=['indicator','value'])
    df = df.set_index('indicator')
    return df

#### RandomForestRegressor

In [ ]:
clf_RDM = {'params' : {'n_estimators':[300],'bootstrap':[True], 'criterion':['mse']},'clf':RandomForestRegressor()}


#### GradientBoostingRegressor

In [ ]:

# clf_gbr = GradientBoostingRegressor()
# params = {'loss' : ['ls', 'lad', 'huber', 'quantile']}

# clf_gbr, Xtrain, ytrain, Xtest, ytest = do_classify(clf_gbr,params,X,y)
# print clf_gbr.score(Xtest,ytest)

#### BayesianRidge


In [ ]:
# clf_br = linear_model.BayesianRidge()
# params = {'alpha_1':[1e-10,1e-11,1e-12,1e-09], 'alpha_2':[1e-09,1e-08],
#        'lambda_1':[1e-08,1e-06,1e-05,1e-04], 'lambda_2':[1e-12,1e-13], 
#           'n_iter':[300,600,1000],
#         'tol':[0.001,0.00001,0.0001,0.01], 'verbose':[False],'compute_score':[False]}

# clf_br, Xtrain, ytrain, Xtest, ytest = do_classify(clf_br,params,X,y)


#### Linear regression

In [ ]:
# clf_lr = LinearRegression()
# clf_lr.fit(Xtrain,ytrain)
# clf_lr.score(Xtest,ytest)

#### Lasso

In [ ]:
clf_LASSO = {'clf' :linear_model.Lasso(),'params' : {'alpha': [0.000001,0.0001,0.001,0.01,0.1,1,10,20,30,40,100,500]}}


#### SGD Regressor

In [ ]:
# clf_sgd = linear_model.SGDRegressor()
# params = {'alpha':[0.000001,0.0001,0.001,0.01,0.1,1,10,20,30,40,100,500]}
# clf_sgd, Xtrain, ytrain, Xtest, ytest = do_classify(clf_sgd,params,X,y)
# compute_forecasting_quality(clf_sgd,X_forecast[:,3:],y_forecast)

#### TPOT Predictor

In [ ]:
clf_TPOT = make_pipeline(
    MaxAbsScaler(),
    make_union(VotingClassifier([("est", LassoLarsCV(normalize=False))]), FunctionTransformer(lambda X: X)),
    RandomForestRegressor(n_estimators=800))

In [ ]:
df_forecasts_baseline = baseline_service.create_baseline(start_forecasts,df_forecasts)
df_forecasts_baseline[df_forecasts_baseline.idbldsite==44]


In [ ]:
def plot_forecasts(axs,i,site_id,site_name,clf_list,X_forecast,y_forecast):
    
    baseline = df_forecasts_baseline[df_forecasts_baseline.idbldsite==site_id]
    ## plot real data
    axs[i].bar(df_forecasts[df_forecasts.idbldsite==site_id].date.values,y_forecast,alpha=0.7,color='grey',align='center')
    
    
    ## plot baseline
    
    axs[i].plot(baseline.date.values,baseline.compensatedin,color='cyan')
    axs[i].set_title(str(site_id),fontsize=15)
#     axs[i].set_axis_bgcolor(BACKGROUND_COLOR)

    for clf in clf_list:
        axs[i].plot(df_forecasts[df_forecasts.idbldsite==site_id].date.values,(clf.predict(X_forecast[:,3:])),alpha=0.9)
    
    for label in axs[i].get_xticklabels():
        label.set_fontname('Arial')
        label.set_fontsize(8)
        label.set_rotation(90)
        
    for label in axs[i].get_yticklabels():
        label.set_fontname('Arial')
        label.set_fontsize(8)
        
   
    axs[0].legend(['Baseline', 'Randomforest','Reality'],loc=0)

In [ ]:
def draw_forecast():
    site_dict = db_service.get_sites_dict()
    sites_ids = site_dict.keys()
    plots_rows_nbr = (len(sites_ids))+1
    print plots_rows_nbr
    fig, axs = plt.subplots(plots_rows_nbr,1, figsize=(20, 80), facecolor='w', edgecolor='k', sharex=False, sharey=False)
    fig.subplots_adjust(hspace = .5, wspace=0.5)
    fig.suptitle("Total In forecasts")
    axs = axs.ravel()
    
    for i,site_id in enumerate(sites_ids):
        print "site_id :" + str(site_id)
        try :
            X,y,fields,X_forecast,y_forecast = create_X_Y_per_site(site_id)

            clf = clf_RDM['clf']
            params = clf_RDM['params']
            clf_rdm, Xtrain, ytrain, Xtest, ytest = do_classify(clf,params,X,y)


            compute_forecasting_quality(clf_rdm,X_forecast[:,3:],y_forecast)
            plot_forecasts(axs,i,site_id,site_dict[site_id][0],[clf_rdm],X_forecast,y_forecast)
        except:
            print "Error"
        

In [ ]:
draw_forecast()